# Multi-Model Inference Pipeline for Peatland Navigation

## Overview
This notebook implements an advanced computer vision pipeline that combines multiple deep learning models to enable comprehensive peatland path analysis and navigation assistance.

## Model Components

### 1. Binary Segmentation (U-Net)
- **Purpose**: Primary path identification
- **Architecture**: U-Net with ResNet34 backbone
- **Output**: Binary masks differentiating paths from surroundings
- **Role**: Initial path proposal generation

### 2. Segment Anything Model (SAM)
- **Purpose**: Path boundary refinement
- **Architecture**: Vision Transformer (ViT)
- **Output**: High-precision segmentation masks
- **Role**: Enhanced path delineation

### 3. Object Detection (YOLOv8)
- **Purpose**: Navigation marker identification
- **Architecture**: YOLOv8 with custom head
- **Output**: Bounding boxes and object classes
- **Role**: Context-aware navigation assistance

### 4. Depth Estimation (MiDaS)
- **Purpose**: Scene depth analysis
- **Architecture**: Lightweight MiDaS
- **Output**: Dense depth maps
- **Role**: Real-world measurements and distance estimation

## Pipeline Integration
The system combines these models to provide:
- Real-time path detection
- Precise boundary segmentation
- Navigation marker recognition
- Path width measurements
- Distance estimation to objects

This comprehensive approach enables robust navigation assistance in challenging peatland environments.

## 1. Dependencies and Framework Integration

### Deep Learning Frameworks
1. **PyTorch (`torch`)**
   - Core deep learning operations
   - GPU acceleration support
   - Model deployment infrastructure
   - Tensor computations

2. **Model-Specific Libraries**
   - **ultralytics**: YOLOv8 implementation and inference
   - **segment-anything**: Meta's SAM framework
   - **segmentation-models-pytorch**: U-Net architecture
   - **MiDaS**: Monocular depth estimation

### Image Processing
1. **Computer Vision**
   - **opencv-python (`cv2`)**
     * Video I/O operations
     * Image processing routines
     * Visualization utilities
   
   - **PIL (Python Imaging Library)**
     * Image format handling
     * Color space conversions
     * Image transformations

2. **Data Augmentation**
   - **albumentations**
     * Image transformations
     * Normalization pipelines
     * Tensor conversion

### Utility Libraries
1. **Data Handling**
   - **numpy**: Numerical operations
   - **pathlib**: Path manipulation
   - **collections.deque**: Temporal smoothing

2. **Progress Monitoring**
   - **tqdm**: Progress bar visualization
   - Process monitoring
   - Time estimation

This comprehensive set of libraries enables efficient implementation of the multi-model inference pipeline.

In [1]:
import torch
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
from ultralytics import YOLO
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
from collections import deque
from segment_anything import sam_model_registry, SamPredictor

/Users/stahlma/.pyenv/versions/peatland_navigation/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. System Configuration and Parameters

### Model Configuration

1. **Path Selection**
   - **Binary Segmentation Model**
     * U-Net checkpoint selection
     * Best performance model identification
     * Weight file location
   
   - **Object Detection Model**
     * YOLOv8 fine-tuned weights
     * Custom class adaptation
     * Inference configuration
   
   - **SAM Model**
     * Pretrained weight selection
     * Architecture specification (ViT-B)
     * Model registry integration

2. **Video Processing Parameters**
   - **Input Configuration**
     * Source video selection
     * Frame rate optimization
     * Resolution handling
   
   - **Output Settings**
     * Destination path configuration
     * Format specifications
     * Quality parameters

3. **Inference Parameters**
   - **Detection Thresholds**
     * Confidence threshold optimization
     * Minimum path area validation
     * False positive filtering
   
   - **Hardware Utilization**
     * Device selection logic
     * GPU/CPU optimization
     * Memory management

4. **Path Analysis Settings**
   - **Smoothing Parameters**
     * Temporal window size
     * Weight distribution
     * Update frequency

   - **Measurement Calibration**
     * Distance scaling factors
     * Camera parameters
     * Real-world unit conversion

These configuration parameters are crucial for:
- Optimal model performance
- Efficient resource utilization
- Accurate measurements
- Reliable real-time processing

In [18]:
# --- Model Paths ---
SEGMENTATION_RUN_NAME = "binary_unet_2025-08-03_22-53-32" 
DETECTION_RUN_NAME = "finetuned" 
SAM_CHECKPOINT_PATH = Path("./training/sam_vit_b.pth") 
SAM_MODEL_TYPE = "vit_b"

# --- Video Configuration ---
VIDEO_FILENAME = "Clip_3_35s.mp4" 
FPS_REDUCTION_FACTOR = 6 # Process 1 frame every N frames (60fps / 6 = 10fps)

# --- Detection Configuration ---
CONFIDENCE_THRESHOLD = 0.5
MIN_PATH_AREA_THRESHOLD = 2000 # Minimum area of the path to be considered valid

# --- Paths and settings derived automatically ---
SEG_MODEL_PATH = Path("./training/metrics/segmentation") / SEGMENTATION_RUN_NAME / "best_binary_model.pth"
DET_MODEL_PATH = Path("./training/metrics/detection") / DETECTION_RUN_NAME / "weights/best.pt"
VIDEO_PATH = Path("./data/video/splits") / VIDEO_FILENAME
OUTPUT_DIR = Path("final_video_output")
OUTPUT_DIR.mkdir(exist_ok=True)
OUTPUT_VIDEO_PATH = OUTPUT_DIR / f"showcase_{VIDEO_FILENAME}"

if torch.cuda.is_available(): DEVICE = "cuda"
elif torch.backends.mps.is_available(): DEVICE = "mps"
else: DEVICE = "cpu"

print(f"Device: {DEVICE}")
print(f"Loading Segmentation Model: {SEG_MODEL_PATH}")
print(f"Loading Detection Model: {DET_MODEL_PATH}")

Device: mps
Loading Segmentation Model: training/metrics/segmentation/binary_unet_2025-08-03_22-53-32/best_binary_model.pth
Loading Detection Model: training/metrics/detection/finetuned/weights/best.pt


## 3. Model Initialization and Architecture Setup

### 1. Binary Segmentation (U-Net)

**Architecture Components:**
- ResNet34 encoder for feature extraction
- Skip connections for detail preservation
- Binary classification head
- Custom preprocessing pipeline

**Configuration Details:**
- Input channels: 3 (RGB)
- Output classes: 2 (path/non-path)
- Image size: 480x640
- Normalization: ImageNet statistics

### 2. Segment Anything Model (SAM)

**Implementation Details:**
- ViT-B architecture variant
- Point-prompt based refinement
- Multi-mask output capability
- High-precision boundary detection

**Initialization Process:**
- Model registry integration
- Weight loading and verification
- Device optimization
- Predictor setup

### 3. Object Detection (YOLOv8)

**Model Configuration:**
- Fine-tuned weights loading
- Class mapping verification
- Confidence threshold setup
- Non-maximum suppression settings

**Detection Classes:**
- Bench markers
- Navigation cones
- Class ID mapping for visualization

### 4. Depth Estimation (MiDaS)

**Model Variant:**
- Small architecture for efficiency
- Real-time processing capability
- Monocular depth estimation
- Custom transform pipeline

**Processing Setup:**
- Input normalization
- Resolution adaptation
- Output scaling
- Device optimization

Each model is configured for optimal performance while maintaining real-time processing capabilities in the integrated pipeline.

In [11]:
# --- Segmentation U-Net ---
seg_model = smp.Unet("resnet34", encoder_weights=None, in_channels=3, classes=2).to(DEVICE)
seg_model.load_state_dict(torch.load(SEG_MODEL_PATH, map_location=torch.device(DEVICE)))
seg_model.eval()
seg_transform = A.Compose([
    A.Resize(height=480, width=640),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0),
    ToTensorV2(),
])
print("Segmentation model loaded.")

# --- SAM ---
sam = sam_model_registry[SAM_MODEL_TYPE](checkpoint=SAM_CHECKPOINT_PATH)
sam.to(device=DEVICE)
sam_predictor = SamPredictor(sam)
print("SAM model loaded.")

# --- Detection YOLOv8 ---
detection_model = YOLO(DET_MODEL_PATH)
BENCH_CLASS_ID = [k for k, v in detection_model.names.items() if v == 'bench'][0]
CONE_CLASS_ID = [k for k, v in detection_model.names.items() if v == 'cone'][0]
print("Detection model loaded.")

# --- Depth MiDaS ---
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS_small", trust_repo=True)
midas.to(DEVICE)
midas.eval()
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms", trust_repo=True)
midas_transform = midas_transforms.small_transform
print("Depth model loaded.")

Segmentation model loaded.
SAM model loaded.
Detection model loaded.


Using cache found in /Users/stahlma/.cache/torch/hub/intel-isl_MiDaS_master
/Users/stahlma/.pyenv/versions/peatland_navigation/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Loading weights:  None


Using cache found in /Users/stahlma/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


Depth model loaded.


Using cache found in /Users/stahlma/.cache/torch/hub/intel-isl_MiDaS_master


## 4. Core Processing Functions

### Path Analysis Functions

1. **Path Mask Generation (`get_smoothed_path_mask`)**
   
   **Purpose:**
   - Generate temporally consistent path segmentation
   - Apply U-Net for initial prediction
   - Implement weighted temporal smoothing
   
   **Implementation Details:**
   - Moving average over multiple frames
   - Weighted contribution of historical predictions
   - Resolution matching and interpolation

2. **Depth Map Generation (`get_depth_map`)**
   
   **Purpose:**
   - Generate dense depth maps
   - Enable distance measurements
   - Support path width estimation
   
   **Technical Components:**
   - MiDaS inference pipeline
   - Resolution adaptation
   - Depth map normalization
   - Output calibration

3. **Path Point Extraction (`mask_to_prompt_points`)**
   
   **Purpose:**
   - Extract key points for SAM refinement
   - Validate path detection confidence
   - Generate prompt coordinates
   
   **Processing Steps:**
   - Contour detection and filtering
   - Area-based validation
   - Centroid calculation
   - Multi-point selection

4. **Width Measurement (`estimate_width_in_meters`)**
   
   **Purpose:**
   - Convert pixel measurements to real-world units
   - Account for perspective effects
   - Provide accurate path width estimates
   
   **Implementation Details:**
   - Pinhole camera model implementation
   - Focal length calculation
   - Distance-based scaling
   - Calibration factor application

### Function Integration

These utility functions form a coordinated processing pipeline:
1. Initial path detection (U-Net)
2. Temporal consistency (Smoothing)
3. Boundary refinement (SAM)
4. Measurement calculation (Depth + Geometry)

The functions are designed for:
- Efficient processing
- Robust error handling
- Real-time performance
- Accurate measurements

In [12]:
PATH_COLOR_BGR = (227, 124, 240)  # A pleasing light purple
PATH_CLASS_ID = 1
DISTANCE_CALIBRATION_FACTOR = 500 

def get_smoothed_path_mask(frame, logits_history):
    """Gets U-Net prediction and applies a weighted moving average."""
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_tensor = seg_transform(image=image_rgb)['image'].unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        current_logits = seg_model(image_tensor)
    
    logits_history.append(current_logits)
    weights = torch.linspace(0.1, 1.0, len(logits_history)).to(DEVICE)
    weights = weights / weights.sum()
    
    weighted_sum = torch.zeros_like(current_logits)
    for i, logits in enumerate(logits_history):
        weighted_sum += logits * weights[i]
        
    preds = torch.argmax(weighted_sum, dim=1).squeeze(0).cpu().numpy().astype(np.uint8)
    return cv2.resize(preds, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)

def get_depth_map(frame):
    """Gets MiDaS depth prediction."""
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_batch = midas_transform(img_rgb).to(DEVICE)
    with torch.no_grad():
        prediction = midas(input_batch)
    return torch.nn.functional.interpolate(
        prediction.unsqueeze(1), size=img_rgb.shape[:2], mode="bicubic", align_corners=False
    ).squeeze().cpu().numpy()

def mask_to_prompt_points(mask, max_points=3):
    """Finds the largest contours in the U-Net mask and returns their center points."""
    contours, _ = cv2.findContours((mask == PATH_CLASS_ID).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours: return None, 0
    
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    largest_contour_area = cv2.contourArea(sorted_contours[0])
    
    points = []
    for contour in sorted_contours[:max_points]:
        M = cv2.moments(contour)
        if M["m00"] > 0:
            points.append([int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])])
            
    return np.array(points) if points else None, largest_contour_area

def estimate_width_in_meters(pixel_width, distance_m, frame_width, fov_degrees=70):
    """Estimates real-world width using the pinhole camera model."""
    # Calculate the camera's focal length in pixels
    fov_radians = np.deg2rad(fov_degrees)
    focal_length_pixels = (frame_width / 2) / np.tan(fov_radians / 2)
    
    # Calculate the size of one pixel in meters at the given distance
    meters_per_pixel = distance_m / focal_length_pixels
    
    # Estimate the real-world width
    width_meters = pixel_width * meters_per_pixel * 1.5  # Adjust factor based on visual calibration
    return width_meters

## 5. Video Processing Pipeline

### Input Processing Framework

1. **Video Stream Management**
   - Frame extraction with controlled FPS
   - Resolution maintenance
   - Format standardization
   - Progress monitoring

2. **Frame Processing**
   - Color space conversions
   - Resolution adaptation
   - Batch preparation
   - Memory management

### Multi-Model Inference Pipeline

1. **Path Segmentation Process**
   
   **Initial Detection:**
   - U-Net inference
   - Confidence validation
   - Area thresholding
   - Temporal consistency check
   
   **Refinement Stage:**
   - SAM prompt generation
   - Boundary refinement
   - IoU-based mask selection
   - Temporal smoothing

2. **Object Detection Integration**
   
   **Detection Process:**
   - YOLOv8 inference
   - Confidence filtering
   - Class identification
   - Non-maximum suppression
   
   **Distance Analysis:**
   - Depth map integration
   - Distance calculation
   - Real-world scaling
   - Measurement annotation

3. **Measurement Pipeline**
   
   **Path Analysis:**
   - Width measurement points
   - Depth integration
   - Real-world conversion
   - Temporal stability
   
   **Object Relations:**
   - Distance to path
   - Inter-object distances
   - Spatial relationships
   - Navigation context

### Visualization Framework

1. **Path Visualization**
   - Contour overlay
   - Transparency management
   - Color coding
   - Width indicators

2. **Object Annotation**
   - Bounding box rendering
   - Class labels
   - Confidence scores
   - Distance measurements

3. **Measurement Display**
   - Width measurements
   - Distance annotations
   - Unit conversion
   - Text placement

### Pipeline Integration

The system maintains:
- Frame-to-frame consistency
- Real-time processing capability
- Memory efficiency
- Robust error handling

This comprehensive pipeline enables detailed scene understanding and measurement for navigation assistance.

In [19]:
cap = cv2.VideoCapture(str(VIDEO_PATH))
if not cap.isOpened():
    print(f"Error opening video file {VIDEO_PATH}")
else:
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(OUTPUT_VIDEO_PATH), fourcc, fps, (frame_width, frame_height))

    last_refined_mask, last_boxes, last_depth_map, last_known_points = None, [], None, None
    frame_count = 0
    logits_history = deque(maxlen=10)

    for _ in tqdm(range(total_frames), desc="Creating final video"):
        ret, frame = cap.read()
        if not ret: break

        if frame_count % FPS_REDUCTION_FACTOR == 0:
            unet_mask = get_smoothed_path_mask(frame, logits_history)
            current_points, path_area = mask_to_prompt_points(unet_mask)
            
            # --- UPDATED: Confidence check for path prediction ---
            if path_area > MIN_PATH_AREA_THRESHOLD:
                last_known_points = current_points
            else: # If path is too small or non-existent, don't use a prompt
                last_known_points = None

            if last_known_points is not None:
                sam_predictor.set_image(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                masks_sam, _, _ = sam_predictor.predict(
                    point_coords=last_known_points,
                    point_labels=np.ones(len(last_known_points)),
                    multimask_output=True,
                )
                
                best_iou, best_mask = -1, None
                unet_path_mask_bool = (unet_mask == PATH_CLASS_ID)
                for mask in masks_sam:
                    intersection = np.logical_and(unet_path_mask_bool, mask).sum()
                    union = np.logical_or(unet_path_mask_bool, mask).sum()
                    iou = intersection / union if union > 0 else 0
                    if iou > best_iou:
                        best_iou, best_mask = iou, mask
                last_refined_mask = best_mask
            else:
                last_refined_mask = None
            
            last_boxes = detection_model(frame, verbose=False)[0].boxes
            last_depth_map = get_depth_map(frame)

        annotated_frame = frame.copy()
        
        if last_refined_mask is not None:
            contours, _ = cv2.findContours(last_refined_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(annotated_frame, contours, -1, PATH_COLOR_BGR, 3)

            width_measure_y = int(frame_height * 0.8)
            line = last_refined_mask[width_measure_y, :]
            path_pixels = np.where(line)[0]
            # if len(path_pixels) > 1 and last_depth_map is not None:
            #     x_start, x_end = path_pixels[0], path_pixels[-1]
            #     pixel_width = x_end - x_start
                
            #     depth_at_line = np.median(last_depth_map[width_measure_y, x_start:x_end])
            #     distance_m = DISTANCE_CALIBRATION_FACTOR / depth_at_line
                
            #     path_width_meters = estimate_width_in_meters(pixel_width, distance_m, frame_width)

            #     cv2.line(annotated_frame, (x_start, width_measure_y), (x_end, width_measure_y), (255, 255, 0), 3)
            #     label = f"Path Width: {path_width_meters:.2f}m"
            #     cv2.putText(annotated_frame, label, (x_start, width_measure_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

        if last_boxes is not None and last_depth_map is not None:
            for box in last_boxes:
                if box.conf >= CONFIDENCE_THRESHOLD:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    class_id = int(box.cls[0])
                    class_name = detection_model.names[class_id]
                    
                    label = f"{class_name} ({box.conf[0]:.2f})"
                    color = (0, 255, 0)

                    box_depth = last_depth_map[y1:y2, x1:x2]
                    if box_depth.size > 0:
                        distance = DISTANCE_CALIBRATION_FACTOR / np.median(box_depth)
                        label += f" | Dist: {distance:.1f}m"

                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(annotated_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        out.write(annotated_frame)
        frame_count += 1
        
    cap.release()
    out.release()
    print(f"\nFinal showcase video saved to: {OUTPUT_VIDEO_PATH}")


Creating final video: 100%|██████████| 1065/1065 [02:12<00:00,  8.01it/s]


Final showcase video saved to: final_video_output/showcase_Clip_3_35s.mp4
